# Advanced Retrieval Strategies
## ReRank

### FlagEmbeddingReranker
這是用於在本地執行開源重新排序（reranker）模型的 **LlamaIndex 整合類別**。

**運作方式：**  
它是對 BAAI（北京智源人工智慧研究院）模型套件的封裝。該類別採用 **Cross-Encoder（交叉編碼器）架構**，也就是同時處理查詢（query）與文件（document），以計算「真正的」相關性分數，而不是比較預先計算好的向量。  

**最適合對象：**  
希望在自有硬體（GPU）上執行高品質重新排序、且不想支付 API 呼叫費用的開發者。

---

**BAAI/bge-reranker-large**

這是目前全球最受歡迎的開源重新排序模型之一。

**主要優勢：**  
它在 MTEB（Massive Text Embedding Benchmark，大規模文字嵌入基準測試）中長期名列前茅。「Large」版本（約 5.6 億參數）在英文與中文任務上都具有極高的準確度。  

**取捨：**  
由於它是 Cross-Encoder 架構，其速度明顯慢於初始的向量搜尋。因此，通常只會將前 10–50 筆文件傳入該模型進行重新排序。

**目前狀態：**  
較新的 **BGE-Reranker-v2-m3**（近期發布）功能更加全面，支援多語言並可處理極長的上下文內容。


In [ ]:
pip install git+https://github.com/FlagOpen/FlagEmbedding.git

In [ ]:
import os

# os.chdir("../")

from llama_index.llms.openai import OpenAI
# from llama_index.indices.postprocessor import LLMRerank 
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker

from initialization import credential_init

credential_init()

llm = OpenAI("gpt-4o-mini")
reranker = FlagEmbeddingReranker(
    top_n=10,
    model="BAAI/bge-reranker-large",
    use_fp16=False
)
# reranker = LLMRerank(choice_batch_size=5, top_n=5, llm=llm)

In [ ]:
import wikipediaapi

wiki_titles = ["鋼之鍊金術師", "一拳超人", "ONE_PIECE", "東京喰種"]
wiki_metadatas = {
    "鋼之鍊金術師": {
        "author": "荒川弘",
    },
    "一拳超人": {
        "author": "ONE",
    },
    "ONE_PIECE": {
        "author": "尾田榮一郎",
    },
    "東京喰種": {
        "author": "石田翠",
    },
}

wiki_wiki = wikipediaapi.Wikipedia(user_agent='AI Tutorial(mengchiehling@gmail.com)', language='zh-tw')

從Wikipedia下載數據

In [ ]:
from pathlib import Path

from llama_index.core import SimpleDirectoryReader

data_path = Path("week_3/data")

for title in wiki_titles:
    page = wiki_wiki.page(title)
    wiki_text = page.text

    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path /f"{title}.txt", "w", encoding="utf-8") as fp:
        fp.write(wiki_text)

# Load all wiki documents
docs_dict = {}
for wiki_title in wiki_titles:
    doc = SimpleDirectoryReader(
        input_files=[data_path/f"{wiki_title}.txt"]
    ).load_data()[0]

    doc.metadata.update(wiki_metadatas[wiki_title])
    docs_dict[wiki_title] = doc

documents = [docs_dict[wiki_title] for wiki_title in wiki_titles]

將上週的內容copy/paste下來，建立VectorStoreIndex

In [ ]:
import faiss
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.callbacks import LlamaDebugHandler, CallbackManager

d = 1024 # 必須與 embedding model 的輸出維度一致

faiss_index = faiss.IndexFlatL2(d)
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

callback_manager = CallbackManager([LlamaDebugHandler()])

index = VectorStoreIndex(
    [],
    storage_context=storage_context,
    transformations=[SentenceSplitter.from_defaults()],
    callback_manager=callback_manager,
    embed_model=embed_model
)

# add documents to index
for wiki_title in wiki_titles:
    index.insert(docs_dict[wiki_title])

In [ ]:
print(len(index.index_struct.nodes_dict))

In [ ]:
rerank_query_engine = index.as_query_engine(
    similarity_top_k=50,
    node_postprocessors=[
        reranker
    ]
)

basic_query_engine = index.as_query_engine(
    similarity_top_k=5,
)

In [ ]:
rerank_response = rerank_query_engine.query("英雄")

In [ ]:
retrieved_nodes = rerank_response.source_nodes

In [ ]:
len(retrieved_nodes)

In [ ]:
retrieved_nodes[:5]

In [ ]:
basic_response = basic_query_engine.query("英雄")

In [ ]:
retrieved_nodes = basic_response.source_nodes

In [ ]:
retrieved_nodes

### GPTReranker

RankGPT 是一種**列表式重排序（Listwise Reranking）**策略，利用大型語言模型（如 GPT-4o 或 Llama 3）來對文件進行排序。

- 運作原理： 它並非逐一為文件評分，而是將查詢（Query）與一整組文件列表同時輸入至大型語言模型（LLM），並詢問：「這些文件中哪些最相關？請按順序輸出編號。」
- 核心優勢： 它能發揮前沿 LLM 的完整推理能力，比起小型的編碼器模型（Encoder-only models），它更能理解語意細微的差別、複雜邏輯以及使用者的意圖。
- 權衡考量： 成本較高（API Token 消耗）且延遲較長（需等待 LLM 生成結果）。它通常被視為高標準檢索流程中的「最後磨光（Final Polish）」步驟。

In [ ]:
# !pip install llama-index-postprocessor-rankgpt-rerank

In [ ]:
from llama_index.postprocessor.rankgpt_rerank import RankGPTRerank

reranker = RankGPTRerank(llm=llm)

In [ ]:
rerank_query_engine = index.as_query_engine(
    similarity_top_k=50,
    node_postprocessors=[
        reranker
    ]
)

basic_query_engine = index.as_query_engine(
    similarity_top_k=5,
)

In [ ]:
rerank_response = rerank_query_engine.query("英雄")
retrieved_nodes = rerank_response.source_nodes

In [ ]:
retrieved_nodes[:5]

In [ ]:
basic_response = basic_query_engine.query("英雄")
retrieved_nodes = basic_response.source_nodes

In [ ]:
retrieved_nodes

### Cohere Rerank 4

Cohere Rerank 4 (正式名稱為 Rerank v4.0) 是 Cohere 於 2025 年底推出的最新一代重排序模型。與前代（v3.5）相比，它在效能、上下文長度和企業級應用上都有顯著的突破，被視為目前生產環境中最領先的託管型重排序解決方案之一。

以下是 Rerank 4 的關鍵特性：

1. 雙版本模型策略
Rerank 4 採取了與 LLM 類似的策略，提供了兩款針對不同需求優化的版本：

Rerank 4 Pro: 最強性能版本，專為追求極致準確度的複雜任務設計（如金融、法律、醫療）。在多項基準測試中其排名均位居前二。

Rerank 4 Fast: 效能平衡版本，在維持比 v3.5 更高準確度的同時，大幅降低了延遲（約比 Pro 快 30-40%），適合高流量與即時性需求。

2. 上下文窗口巨大飛躍 (32k Tokens)
從 4k 到 32k： 相比前代 v3.5 的 4096 tokens，Rerank 4 的上下文長度提升了 8 倍。

處理長文件： 這意味著它現在可以一次性「閱讀」約 50 頁的文件，而不需要將文件切成碎片。這對於需要跨章節理解內容的 RAG 系統來說至關重要。

3. 多語言與跨語言能力
支持 100+ 種語言： 在中文、日文、德文、阿拉伯文等主流語言的表現達到了業界領先水平。

跨語言檢索： 它能極其精準地處理「英文查詢、中文文件」這類跨語言的相關性匹配。

4. 結構化數據與推理能力
原生支持 YAML/JSON： 它可以直接對結構化數據進行排序，而不僅僅是純文本。

更強的推理： 針對需要邏輯推理的查詢（例如：需要綜合多個條件的搜尋），Rerank 4 的理解能力顯著優於傳統的編碼器模型，表現更接近小型 LLM，但成本與速度更優。

5. 易於整合
API 接口： 只需幾行程式碼即可接入（支援 LlamaIndex、LangChain 等框架）。

雲端支持： 已整合進 Azure AI Foundry、AWS SageMaker 等主流企業雲平台，滿足安全性與合規性要求。

總結來說： 如果您正在開發一個企業級 RAG 系統，且對多語言支持、長文件處理或金融/醫療數據精準度有很高要求，Cohere Rerank 4 Pro 是目前的最佳選擇；如果您更在意成本與回覆速度，Rerank 4 Fast 則提供了極佳的性價比。

In [ ]:
from llama_index.postprocessor.cohere_rerank import CohereRerank

# api_key = os.environ["COHERE_API_KEY"]
# cohere_rerank = CohereRerank(api_key=api_key, top_n=2)

In [ ]:
CohereRerank?

## Hybrid Retriever

### BM25

使用英文文本。因為LlamaIndex 的 BM25在處理中文文本上需要高度客製化 

In [9]:
from pathlib import Path

import wikipediaapi


wiki_titles = ["Anthropic", "OpenAI", "XAI_(company)", "Tesla,_Inc."]
wiki_metadatas = {
    "Anthropic": {
        "headquarter": "San Francisco",
        "industry": "Artificial intelligence"
    },
    "OpenAI": {
        "headquarter": "San Francisco",
        "industry": "Artificial intelligence"
    },
    "XAI_(company)": {
        "headquarter": "Palo Alto",
        "industry": "Technology"
    },
    "Tesla,_Inc.": {
        "headquarter": "Austin",
        "industry": "Automotive"
    },
}


wiki_wiki = wikipediaapi.Wikipedia(user_agent='AI Tutorial(mengchiehling@gmail.com)', language='en')

In [10]:
for title in wiki_titles:
    page = wiki_wiki.page(title)
    wiki_text = page.text

    data_path = Path("data")
    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path / f"{title}.txt", "w", encoding="utf-8") as fp:
        fp.write(wiki_text)

In [11]:
documents = []

for wiki_title in wiki_titles:
    doc = SimpleDirectoryReader(
        input_files=[f"data/{wiki_title}.txt"]
    ).load_data()[0]

    doc.metadata.update(wiki_metadatas[wiki_title])
    documents.append(doc)

In [12]:
from llama_index.core import Document
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter

# initialize node parser
splitter = SentenceSplitter(chunk_size=512)

nodes = splitter.get_nodes_from_documents(documents)

In [13]:
len(nodes)

117

One option is to create the BM25Retriever directly from nodes, and save to and from disk.

In [14]:
import Stemmer

from llama_index.retrievers.bm25 import BM25Retriever

# We can pass in the index, docstore, or list of nodes to create the retriever
bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=5,
    # Optional: We can pass in the stemmer and set the language for stopwords
    # This is important for removing stopwords and stemming the query + text
    # The default is english for both
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)

2026-01-06 23:13:18,384 - DEBUG - Building index from IDs objects


Here, we cover using a BM25Retriever with a docstore to hold your nodes. The advantage here is that the docstore can be remote (mongodb, redis, etc.)

In [15]:
# initialize a docstore to store nodes
# also available are mongodb, redis, postgres, etc for docstores
from llama_index.core.storage.docstore import SimpleDocumentStore

docstore = SimpleDocumentStore()
docstore.add_documents(nodes)

# We can pass in the index, docstore, or list of nodes to create the retriever
bm25_retriever = BM25Retriever.from_defaults(
    docstore=docstore,
    similarity_top_k=5,
    # Optional: We can pass in the stemmer and set the language for stopwords
    # This is important for removing stopwords and stemming the query + text
    # The default is english for both
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)

2026-01-06 23:27:48,984 - DEBUG - Building index from IDs objects


In [17]:
# will retrieve context from specific companies
retrieved_nodes = bm25_retriever.retrieve(
    "artiticial intelligence"
)

In [18]:
len(retrieved_nodes)

5

In [19]:
retrieved_nodes[0]

NodeWithScore(node=TextNode(id_='208c0afe-045a-49bf-b8f7-0aab99c047d9', embedding=None, metadata={'file_path': 'data\\Anthropic.txt', 'file_name': 'Anthropic.txt', 'file_type': 'text/plain', 'file_size': 14618, 'creation_date': '2026-01-06', 'last_modified_date': '2026-01-06', 'headquarter': 'San Francisco', 'industry': 'Artificial intelligence'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='c53d3e3f-a06c-4e86-bfe6-7055882c7feb', node_type='4', metadata={'file_path': 'data\\Anthropic.txt', 'file_name': 'Anthropic.txt', 'file_type': 'text/plain', 'file_size': 14618, 'creation_date': '2026-01-06', 'last_modified_date': '2026-01-06', 'headquarter': 'San Francisco', 'industry': 'Artificial intelligence'}, ha

### BM25 Retriever + MetadataFiltering

In [20]:
# Define metadata filters
from llama_index.core.vector_stores.types import (
    MetadataFilters,
    MetadataFilter,
    FilterOperator,
    FilterCondition,
)

filters = MetadataFilters(
    filters=[
        MetadataFilter(
            key="headquarter",
            value="San Francisco",
            operator=FilterOperator.EQ,
        )
    ],
    condition=FilterCondition.AND,
)

In [21]:
retrieved_nodes = BM25Retriever.from_defaults(
    docstore=docstore,
    similarity_top_k=3,
    filters=filters,  # Add filters here
    stemmer=Stemmer.Stemmer("english"),
    language="english",
).retrieve("Hello, world!")

2026-01-06 23:32:24,158 - DEBUG - Building index from IDs objects


In [22]:
!pip install matplotlib

  Using cached contourpy-1.3.2-cp310-cp310-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.9-cp310-cp310-win_amd64.whl.metadata (6.4 kB)
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.1 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.1 MB 2.4 MB/s eta 0:00:04
   ------ --------------------------------- 1.3/8.1 MB 2.4 MB/s eta 0:00:03
   ---------- ----------------------------- 2.1/8.1 MB 2.9 MB/s eta 0:00:03
   -------------- ------------------------- 2.9/8.1 MB 3.3 MB/s eta 0:00:02
   -------------------- ------------------- 4.2/8.1 MB 3.8 MB/s eta 0:00:02
   ------------------------- -------------- 5.2/8.1 MB 4.1 MB/s eta 0:00:01
   -------------------------------- ------- 6.6/8.1 MB 4.4 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 4.8 MB/s  0:00:01
Using cached contourpy-1.3.2-cp310-cp310-w

### Queries

**Company Background & Positioning**

> Best for testing keyword matching + semantic understanding

- What is OpenAI and what is its core mission?
- How does Anthropic position itself in the AI safety landscape?
- What company is xAI and why was it founded?
- What is Tesla primarily known for besides electric vehicles?

---

**Founders & Key Figures**

> Strong keyword signals (names, dates) with semantic reinforcement

- Who founded OpenAI and when?
- Who are the founders of Anthropic?
- What is Elon Musk’s involvement in xAI?
- How is Elon Musk connected to Tesla?

---

**Mission, Goals, and Philosophy**

> Conceptual questions where dense retrieval excels

- What does OpenAI mean by artificial general intelligence?
- What is Anthropic’s approach to building safe AI systems?
- What problem is xAI trying to solve?
- What is Tesla’s long-term vision for sustainable energy?

---

**Products & Technologies**

> Hybrid retrieval works especially well for mixed factual + semantic queries

- What products or models has OpenAI released?
- What is Anthropic’s Claude model?
- Does xAI develop large language models?
- What technologies does Tesla develop besides cars?

---

**Comparison Queries**

> Ideal for demonstrating QueryFusionRetriever and multi-document reasoning

- How is OpenAI different from Anthropic?
- Compare the goals of OpenAI and xAI.
- How do AI companies like OpenAI and Anthropic differ from Tesla?
- Which of these companies focuses on AI safety?

---

**Advanced / RAG Demo Queries (Recommended)**

> Excellent for end-to-end RetrieverQueryEngine demonstrations

- Which companies in the documents are focused on artificial intelligence research?
- Which organization emphasizes AI safety and alignment?
- What companies were founded by Elon Musk?
- Summarize the missions of OpenAI, Anthropic, xAI, and Tesla.

In [23]:
from llama_index.core.response.notebook_utils import display_source_node

retrieved_nodes = BM25Retriever.from_defaults(
    docstore=docstore,
    similarity_top_k=5,
    filters=filters,  # Add filters here
    stemmer=Stemmer.Stemmer("english"),
    language="english",
).retrieve("Hello, world!")

for node in retrieved_nodes:
    display_source_node(node, source_length=5000)

2026-01-06 23:33:30,917 - DEBUG - Building index from IDs objects


**Node ID:** 91e4866c-c38c-4185-8af7-3b2dab7989ee<br>**Similarity:** 0.8953179717063904<br>**Text:** OpenAI subsequently began a $50 million fund to support nonprofit and community organizations.
In April 2025, OpenAI raised $40 billion at a $300 billion post-money valuation, which was the highest-value private technology deal in history. The financing round was led by SoftBank, with other participants including Microsoft, Coatue, Altimeter and Thrive.
In July 2025, the company reported annualized revenue of $12 billion. This was an increase from $3.7 billion in 2024, which was driven by ChatGPT subscriptions, which reached 20 million paid subscribers by April 2025, up from 15.5 million at the end of 2024, alongside a rapidly expanding enterprise customer base that grew to five million business users.
The company cash burn remains high due to the intensive computational costs required to train and run large language models. It projects an $8 billion operating loss in 2025. OpenAI reports revised long-term spending projections totaling approximately $115 billion through 2029, with annual expenditures projected to escalate significantly, reaching $17 billion in 2026, $35 billion in 2027, and $45 billion in 2028. These expenditures are primarily allocated toward expanding compute infrastructure, developing proprietary AI chips, constructing data centers, and funding intensive model training programs, with more than half of the spending through the end of the decade expected to support research-intensive compute for model training and development.
The company's financial strategy reflects a strategy of prioritizing market expansion and technological advancement over near-term profitability, with OpenAI targeting cash flow positive operations by 2029 and projecting revenue of approximately $200 billion by 2030. This aggressive spending trajectory underscores both the enormous capital requirements of scaling cutting-edge AI technology and OpenAI's commitment to maintaining its position as a leader in the artificial intelligence industry.
In October 2025, OpenAI completed an employee share sale of up to $10 billion to existing investors which valued the company at $500 billion. The deal values OpenAI as the most valuable privately owned company in the world—surpassing SpaceX as the world's most valuable private company.<br>

**Node ID:** a5d167e7-29dd-4a45-9905-dd0893d38aee<br>**Similarity:** 0.8840523958206177<br>**Text:** These expenditures are primarily allocated toward expanding compute infrastructure, developing proprietary AI chips, constructing data centers, and funding intensive model training programs, with more than half of the spending through the end of the decade expected to support research-intensive compute for model training and development.
The company's financial strategy reflects a strategy of prioritizing market expansion and technological advancement over near-term profitability, with OpenAI targeting cash flow positive operations by 2029 and projecting revenue of approximately $200 billion by 2030. This aggressive spending trajectory underscores both the enormous capital requirements of scaling cutting-edge AI technology and OpenAI's commitment to maintaining its position as a leader in the artificial intelligence industry.
In October 2025, OpenAI completed an employee share sale of up to $10 billion to existing investors which valued the company at $500 billion. The deal values OpenAI as the most valuable privately owned company in the world—surpassing SpaceX as the world's most valuable private company.

Firing of Altman
On November 17, 2023, Sam Altman was removed as CEO when its board of directors (composed of Helen Toner, Ilya Sutskever, Adam D'Angelo and Tasha McCauley) cited a lack of confidence in him. Chief Technology Officer Mira Murati took over as interim CEO. Greg Brockman, the president of OpenAI, was also removed as chairman of the board and resigned from the company's presidency shortly thereafter. Three senior OpenAI researchers subsequently resigned: director of research and GPT-4 lead Jakub Pachocki, head of AI risk Aleksander Mądry, and researcher Szymon Sidor.
On November 18, 2023, there were reportedly talks of Altman returning as CEO amid pressure placed upon the board by investors such as Microsoft and Thrive Capital, who objected to Altman's departure. Although Altman himself spoke in favor of returning to OpenAI, he has since stated that he considered starting a new company and bringing former OpenAI employees with him if talks to reinstate him didn't work out. The board members agreed "in principle" to resign if Altman returned. On November 19, 2023, negotiations with Altman to return failed and Murati was replaced by Emmett Shear as interim CEO.<br>

**Node ID:** 944c28f3-46a1-4dd8-b75f-b344c1351942<br>**Similarity:** 0.6372473239898682<br>**Text:** Anthropic PBC is an American artificial intelligence (AI) company founded in 2021. It has developed a family of large language models (LLMs) named Claude. The company researches and develops AI to "study their safety properties at the technological frontier" and use this research to deploy safe models for the public.
Anthropic was founded by former members of OpenAI, including siblings Daniela Amodei and Dario Amodei, who serve  as president and CEO respectively. In September 2023, Amazon announced an investment of up to $4 billion. Google committed $2 billion the next month. As of November 2025, Anthropic is valued at over $350 billion.

History
Founding and early development (2021–2022)
Anthropic was founded in 2021 by seven former employees of OpenAI, including siblings Daniela Amodei and Dario Amodei, the latter of whom was OpenAI's Vice President of Research.
In April 2022, Anthropic announced it had received $580 million in funding, including a $500 million investment from FTX under the leadership of Sam Bankman-Fried.
In the summer of 2022, Anthropic finished training the first version of Claude but did not release it, citing the need for further internal safety testing and a desire to avoid initiating a potentially hazardous race to develop increasingly powerful AI systems.

Major investments
In September 2023, Amazon announced a partnership with Anthropic. Amazon became a minority stakeholder by initially investing $1.25 billion and planning a total investment of $4 billion. The remaining $2.75 billion was invested in March 2024. In November 2024, Amazon invested another $4 billion, doubling its total investment. As part of the deal, Anthropic uses Amazon Web Services (AWS) as its primary cloud provider and makes its AI models available to AWS customers.
In October 2023, Google invested $500 million in Anthropic and committed to an additional $1.5 billion over time. In March 2025, Google agreed to invest another $1 billion in Anthropic.

Recruitment (2024)
In February 2024, Anthropic hired former Google Books head of partnerships Tom Turvey, and tasked him with obtaining "all the books in the world".<br>

**Node ID:** 0a5eb095-86ce-4b5f-9a09-0ccfaba4d6c6<br>**Similarity:** 0.6270107626914978<br>**Text:** Major investments
In September 2023, Amazon announced a partnership with Anthropic. Amazon became a minority stakeholder by initially investing $1.25 billion and planning a total investment of $4 billion. The remaining $2.75 billion was invested in March 2024. In November 2024, Amazon invested another $4 billion, doubling its total investment. As part of the deal, Anthropic uses Amazon Web Services (AWS) as its primary cloud provider and makes its AI models available to AWS customers.
In October 2023, Google invested $500 million in Anthropic and committed to an additional $1.5 billion over time. In March 2025, Google agreed to invest another $1 billion in Anthropic.

Recruitment (2024)
In February 2024, Anthropic hired former Google Books head of partnerships Tom Turvey, and tasked him with obtaining "all the books in the world". The company then began using destructive book scanning to digitize "millions" of books to train Claude.
In 2024, Anthropic attracted several notable employees from OpenAI, including Jan Leike, John Schulman, and Durk Kingma.

Additional funding and partnerships (2025)
Anthropic raised $3.5 billion in a Series E funding round in March 2025, achieving a post-money valuation of $61.5 billion, led by Lightspeed Venture Partners with participation from several major investors. In March, Databricks and Anthropic announced that Claude would be integrated into the Databricks Data Intelligence Platform.
In May 2025, the company announced Claude 4, introducing both Claude Opus 4 and Claude Sonnet 4 with improved coding capabilities and other new features. It also introduced new API capabilities, including the Model Context Protocol (MCP) connector. The company hosted its inaugural developer conference that month. Also in May, Anthropic launched a web search API that enables Claude to access real-time information from the internet. Claude Code, Anthropic's coding assistant, transitioned from research preview to general availability, featuring integrations with VS Code and JetBrains IDEs and support for GitHub Actions.
In September 2025, Anthropic completed a Series F funding round, raising $13 billion at a post-money valuation of $183 billion.<br>

**Node ID:** c47b0fef-a0a8-46e6-8f5f-d67ded0cb77b<br>**Similarity:** 0.6170979142189026<br>**Text:** In December 2016, OpenAI released "Universe", a software platform for measuring and training an AI's general intelligence across the world's supply of games, websites, and other applications.

Corporate structure
Transition from non-profit
In 2019, OpenAI transitioned from non-profit to "capped" for-profit, with the profit being capped at 100 times any investment. According to OpenAI, the capped-profit model allows OpenAI Global, LLC to legally attract investment from venture funds and, in addition, to grant employees stakes in the company. Many top researchers work for Google Brain, DeepMind, or Facebook, which offer stock options that a nonprofit would be unable to. Before the transition, public disclosure of the compensation of top employees at OpenAI was legally required.
The company then distributed equity to its employees and partnered with Microsoft, announcing an investment package of $1 billion into the company. Since then, OpenAI systems have run on an Azure-based supercomputing platform from Microsoft.
OpenAI Global, LLC then announced its intention to commercially license its technologies. It planned to spend $1 billion "within five years, and possibly much faster". Altman stated that even a billion dollars may turn out to be insufficient, and that the lab may ultimately need "more capital than any non-profit has ever raised" to achieve artificial general intelligence.
The nonprofit, OpenAI, Inc., is the sole controlling shareholder of OpenAI Global, LLC, which, despite being a for-profit company, retains a formal fiduciary responsibility to OpenAI, Inc.'s nonprofit charter. A majority of OpenAI, Inc.'s board is barred from having financial stakes in OpenAI Global, LLC. In addition, minority members with a stake in OpenAI Global, LLC are barred from certain votes due to conflict of interest. Some researchers have argued that OpenAI Global, LLC's switch to for-profit status is inconsistent with OpenAI's claims to be "democratizing" AI.
On February 29, 2024, Elon Musk filed a lawsuit against OpenAI and CEO Sam Altman, accusing them of shifting focus from public benefit to profit maximization—a case OpenAI dismissed as "incoherent" and "frivolous," though Musk later revived legal action against Altman and others in August.<br>

## Hybrid Retriever with BM25 + FAISS（BM25 + FAISS 的混合式檢索器）

### 1️⃣ 內容翻譯（繁體中文）

### 使用 BM25 + FAISS 的混合式檢索器
現在我們將結合 **BM25** 與 **FAISS**，同時進行**稀疏檢索（sparse retrieval）**與**稠密檢索（dense retrieval）**。

檢索結果會透過 **QueryFusionRetriever** 進行整合。

透過這個 retriever，我們可以建立一個完整的 **RetrieverQueryEngine**。

---

### 2️⃣ Hybrid Retriever（混合式檢索器）的好處說明

混合式檢索器結合了 BM25 與向量檢索（如 FAISS）的優點，能有效彌補彼此的限制：

- **提升召回率（Recall）**
  - BM25 擅長精確的關鍵字比對
  - FAISS 擅長捕捉語意相近但用詞不同的內容
  - 兩者結合能找回更多「真正相關」的文件

- **兼顧語意與關鍵字精準度**
  - 對於專有名詞、數字、程式碼等，BM25 表現較佳
  - 對於語意相似、同義改寫的查詢，向量檢索更有優勢

- **降低單一檢索方式的盲點**
  - 僅使用 BM25 可能忽略語意相關但未出現關鍵字的內容
  - 僅使用向量檢索可能在精確字詞匹配上表現不足

- **透過 Query Fusion 提升排序品質**
  - QueryFusionRetriever 可以將多種檢索結果加權、融合與重新排序
  - 提供更穩定且高品質的最終結果

---

### 3️⃣ 總結

**Hybrid Retriever（BM25 + FAISS）** 是一種同時結合  
👉 **關鍵字精準匹配** 與  
👉 **語意理解能力** 的檢索策略。

透過 **QueryFusionRetriever** 將結果整合後，再搭配 **RetrieverQueryEngine**，  
可以打造出更準確、更健壯、也更適合實務應用的 RAG（Retrieval-Augmented Generation）系統。

In [24]:
import faiss
from llama_index.core import StorageContext
from llama_index.vector_stores.faiss import FaissVectorStore

d = 1024 # 必須與 embedding model 的輸出維度一致

faiss_index = faiss.IndexFlatL2(d)
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

2026-01-07 08:46:27,564 - INFO - Loading faiss with AVX2 support.
2026-01-07 08:46:27,717 - INFO - Successfully loaded faiss with AVX2 support.


In [25]:
documents = []

for wiki_title in wiki_titles:
    doc = SimpleDirectoryReader(
        input_files=[f"data/{wiki_title}.txt"]
    ).load_data()[0]

    doc.metadata.update(wiki_metadatas[wiki_title])
    documents.append(doc)

In [26]:
from llama_index.core.node_parser import SentenceSplitter

# initialize node parser
splitter = SentenceSplitter(chunk_size=512)

nodes = splitter.get_nodes_from_documents(documents)

In [27]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")


2026-01-07 08:46:42,270 - INFO - Load pretrained SentenceTransformer: BAAI/bge-m3


In [28]:
from llama_index.core.callbacks import LlamaDebugHandler, CallbackManager

callback_manager = CallbackManager([LlamaDebugHandler()])

In [30]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex(nodes=nodes, 
                         storage_context=storage_context,
                         callback_manager=callback_manager,
                         embed_model=embed_model)

**********
Trace: index_construction
    |_embedding -> 12.555748 seconds
    |_embedding -> 14.325865 seconds
    |_embedding -> 13.783602 seconds
    |_embedding -> 12.726933 seconds
    |_embedding -> 11.72713 seconds
    |_embedding -> 12.662802 seconds
    |_embedding -> 13.018565 seconds
    |_embedding -> 13.692765 seconds
    |_embedding -> 13.844663 seconds
    |_embedding -> 12.957124 seconds
    |_embedding -> 12.878496 seconds
    |_embedding -> 8.88392 seconds
**********


In [32]:
import os 

os.chdir("../")

from llama_index.llms.openai import OpenAI

from initialization import credential_init

credential_init()

llm = OpenAI("gpt-4o-mini")

In [33]:
import nest_asyncio

nest_asyncio.apply()

from llama_index.core.retrievers import QueryFusionRetriever

retriever = QueryFusionRetriever(
    [
        index.as_retriever(similarity_top_k=2),
        BM25Retriever.from_defaults(
            docstore=index.docstore, similarity_top_k=2
        ),
    ],
    num_queries=1,
    use_async=True,
    llm=llm
)

2026-01-07 09:13:05,405 - DEBUG - Building index from IDs objects


In [35]:
nodes = retriever.retrieve("Hello, world!")
for node in nodes:
    display_source_node(node, source_length=5000)

**Node ID:** f7c44124-d09f-4bfb-8f59-13f9090c1c55<br>**Similarity:** 1.1420073509216309<br>**Text:** In December 2016, OpenAI released "Universe", a software platform for measuring and training an AI's general intelligence across the world's supply of games, websites, and other applications.

Corporate structure
Transition from non-profit
In 2019, OpenAI transitioned from non-profit to "capped" for-profit, with the profit being capped at 100 times any investment. According to OpenAI, the capped-profit model allows OpenAI Global, LLC to legally attract investment from venture funds and, in addition, to grant employees stakes in the company. Many top researchers work for Google Brain, DeepMind, or Facebook, which offer stock options that a nonprofit would be unable to. Before the transition, public disclosure of the compensation of top employees at OpenAI was legally required.
The company then distributed equity to its employees and partnered with Microsoft, announcing an investment package of $1 billion into the company. Since then, OpenAI systems have run on an Azure-based supercomputing platform from Microsoft.
OpenAI Global, LLC then announced its intention to commercially license its technologies. It planned to spend $1 billion "within five years, and possibly much faster". Altman stated that even a billion dollars may turn out to be insufficient, and that the lab may ultimately need "more capital than any non-profit has ever raised" to achieve artificial general intelligence.
The nonprofit, OpenAI, Inc., is the sole controlling shareholder of OpenAI Global, LLC, which, despite being a for-profit company, retains a formal fiduciary responsibility to OpenAI, Inc.'s nonprofit charter. A majority of OpenAI, Inc.'s board is barred from having financial stakes in OpenAI Global, LLC. In addition, minority members with a stake in OpenAI Global, LLC are barred from certain votes due to conflict of interest. Some researchers have argued that OpenAI Global, LLC's switch to for-profit status is inconsistent with OpenAI's claims to be "democratizing" AI.
On February 29, 2024, Elon Musk filed a lawsuit against OpenAI and CEO Sam Altman, accusing them of shifting focus from public benefit to profit maximization—a case OpenAI dismissed as "incoherent" and "frivolous," though Musk later revived legal action against Altman and others in August.<br>

**Node ID:** 8a35877b-fcf3-4423-a2f0-583fc9bff598<br>**Similarity:** 1.1394356489181519<br>**Text:** The Model S was also the best-selling plug-in electric car worldwide for the years 2015 and 2016.
On July 15, 2013, Tesla became a NASDAQ-100 company.
Tesla announced the Tesla Autopilot, a driver-assistance system, in 2014. In September that year, all Tesla cars started shipping with sensors and software to support the feature, with what would later be called "hardware version 1".
Tesla entered the energy storage market, unveiling its Tesla Powerwall (home) and Tesla Powerpack (business) battery packs in April 2015. The company received orders valued at $800 million within a week of the unveiling.
Tesla began shipping its third vehicle, the luxury SUV Tesla Model X, in September 2015, which had 25,000 pre-orders at the time.

SolarCity and Model 3 (2016–2018)
Tesla entered the solar installation business in November 2016 with the purchase of SolarCity, in an all-stock $2.6 billion deal. The business was merged with Tesla's existing battery energy storage products division to form the Tesla Energy subsidiary. The deal was controversial because at the time of the acquisition, SolarCity was facing liquidity issues of which Tesla's shareholders were not informed. In February 2017, Tesla Motors changed its name to Tesla, Inc., to better reflect the scope of its expanded business.
Tesla unveiled its first mass-market vehicle in April 2016, the Model 3 sedan. The Model 3 was less expensive than Tesla's previous three vehicles, and within a week, the company received over 325,000 paid reservations. To speed up production and control costs, Tesla invested heavily in robotics and automation to assemble the Model 3, but the robotics actually slowed the production of the vehicles. This led to significant delays and production problems, a period which the company described as "production hell". By the end of 2018, the production problems had been overcome, and the Model 3 became the world's best-selling electric car from 2018 to 2021.
This period of "production hell" put significant financial pressure on Tesla, and during this time it became one of the most shorted companies in the stock market.<br>

**num_queries**：

在這個步驟中，我們會將多個索引融合成一個單一的檢索器（retriever）。這個檢索器同時也會透過產生與原始問題相關的額外查詢來擴充（augment）你的查詢，並彙總所有查詢的結果。

此設定會執行 4 次查詢：一次使用你的原始查詢，並另外產生 3 個新的查詢。

預設情況下，它會使用以下提示（prompt）來產生額外的查詢：

In [37]:
"""
QUERY_GEN_PROMPT = (
    "You are a helpful assistant that generates multiple search queries based on a "
    "single input query. Generate {num_queries} search queries, one on each line, "
    "related to the following input query:\n"
    "Query: {query}\n"
    "Queries:\n"
)
"""

'\nQUERY_GEN_PROMPT = (\n    "You are a helpful assistant that generates multiple search queries based on a "\n    "single input query. Generate {num_queries} search queries, one on each line, "\n    "related to the following input query:\n"\n    "Query: {query}\n"\n    "Queries:\n"\n)\n'

In [38]:
retriever = QueryFusionRetriever(
    [
        index.as_retriever(similarity_top_k=2),
        BM25Retriever.from_defaults(
            docstore=index.docstore, similarity_top_k=2
        ),
    ],
    num_queries=2,
    use_async=True,
    llm=llm
)

2026-01-07 09:26:37,256 - DEBUG - Building index from IDs objects


In [39]:
nodes = retriever.retrieve("Hello, world!")

2026-01-07 09:26:43,582 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [41]:
len(nodes)

2

Now, we can plug our retriever into a query engine to synthesize natural language responses.

In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever)

In [42]:
QueryFusionRetriever?

Init signature:
QueryFusionRetriever(
    retrievers: List[llama_index.core.base.base_retriever.BaseRetriever],
    llm: Union[str, llama_index.core.llms.llm.LLM, ForwardRef('BaseLanguageModel'), NoneType] = None,
    query_gen_prompt: Optional[str] = None,
    mode: llama_index.core.retrievers.fusion_retriever.FUSION_MODES = <FUSION_MODES.SIMPLE: 'simple'>,
    similarity_top_k: int = 2,
    num_queries: int = 4,
    use_async: bool = True,
    verbose: bool = False,
    callback_manager: Optional[llama_index.core.callbacks.base.CallbackManager] = None,
    objects: Optional[List[llama_index.core.schema.IndexNode]] = None,
    object_map: Optional[dict] = None,
    retriever_weights: Optional[List[float]] = None,
) -> None
Docstring:      Base retriever.
File:           c:\users\ling\miniconda3\envs\rag\lib\site-packages\llama_index\core\retrievers\fusion_retriever.py
Type:           ABCMeta
Subclasses:     

### 1. Reciprocal Rerank（RRF）

**模式（Mode）**：`FUSION_MODE.RECIPROCAL_RANK`（預設）

**運作方式（How it works）**：  
這是目前最常見、也最受歡迎的方法。它不會使用 LLM 來對文件進行評分，而是透過一個數學公式，根據文件在多個結果清單中的排名（rank）來重新排序。

**公式（Formula）**：  

$$
score = \sum_{d \in r} \frac{1}{k + rank(d)}
$$

其中，`k` 是一個常數（通常為 60）。

**適用情境（Best for）**：  
通用型的「混合搜尋（Hybrid Search）」，當你希望在向量搜尋與關鍵字搜尋之間取得良好平衡，同時避免額外的 LLM 成本時，非常適合使用。

---

### 2. Relative Score Fusion（相對分數融合）

**模式（Mode）**：`FUSION_MODE.RELATIVE_SCORE`

**運作方式（How it works）**：  
此方法不再關注文件的排名（例如第 1 名、第 2 名），而是直接使用實際的相似度分數。  
它會將來自不同檢索器（通常分數尺度不同）的分數正規化到 0 到 1 的範圍內，接著再對這些分數取平均。

**適用情境（Best for）**：  
當檢索器所提供的信心分數（confidence score）本身就相當可靠時。

---

### 3. Distribution-Based Score Fusion（分佈式分數融合）

**模式（Mode）**：`FUSION_MODE.DIST_BASED_SCORE`

**運作方式（How it works）**：  
這是一種更進階的分數融合方法，會同時考慮結果集中分數的平均值（mean）與標準差（standard deviation）。  
這有助於處理不同檢索器可能產生「分數非常集中」或「分數分佈非常分散」的情況。

---

### 4. Simple Fusion（簡單融合）

**模式（Mode）**：`FUSION_MODE.SIMPLE`

**運作方式（How it works）**：  
這是一種基礎做法，通常只是將結果直接串接（concatenation）或進行簡單加總。  
相較於 RRF，這種方法在實際生產環境中較少使用，但可用於測試基準效能（baseline performance）。
